# Make sure to start local inference server on LM studio before running following cell

In [1]:
import os
import openai
import pandas as pd
import xlsxwriter
import re
import matplotlib.pyplot as plt


# *** TODO *** 
openai.api_base = "http://localhost:1234/v1" #insert your local server URL from LM Studio
openai.api_key = "" # no need for an API key

def chat_with_llm(prompt):
    response = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = [{"role": "user", "content":prompt}]
    )

    return response.choices[0].message.content.strip()

data = pd.read_csv('data_v4.csv') #load in data

In [39]:

role_textgen = """You are a customer of a large commercial bank such as Wells Fargo, J.P. Morgan Chase, Bank of America, or Citibank. You have had bad experiences and will 
now be voicing your frustration online by writing complaints to your commercial bank. Be creative. Don't be afraid to voice your frustration."""

rubric = """There are two criterias that you are creating these complaints on:

the tone harshness score on a scale of 1-3. 1 will be the least harsh, 2 will be a harsh and critical tone and 3 will be extremely harsh and confrontational.

the event harshness score on a scale of 1-3. 1 are mild issues that dont have a significant impact on the customer's experience. Examples are service disruptions, 
temporary delays in transactions, or other service errors. 2 are more significant issues that may have more of an impact on the customer's overall experience with the bank. 
Examples are unapproved charges, contesting transactions, money loss less that $10,000, social security-related activities, and debt collecting activities. 
3 are severe issues that may have significant impact on the customer's overall experience with the bank. Examples include fraudulent charges, fraud-related activities, 
lawsuit, hacking activities, money loss of more than $10,000."""


# Escalation code below:

In [ ]:
def escalate_complaint(dataframe):
    """    
    Input: takes in a dataframe consisting of 3 columns: 
        - Original Complaint
        - Tone Severity
        - Case Severity
    
    Output: produces a csv file with synthetic complaints escalated by both case and tone severity
            returns the dataframe as well  
    """

    new_escalated_df = dataframe.copy()

    # Initialize columns for escalated complaints
    new_escalated_df['Sentiment Analysis - Original Tone'] = ''
    new_escalated_df['Sentiment Analysis - Original Case'] = ''
    new_escalated_df['Escalated Complaint (Case Increased)'] = ''
    new_escalated_df['Tone Severity - Constant'] = ''
    new_escalated_df['Case Severity - Increased'] = ''
    new_escalated_df['Sentiment Analysis - Complaint 1 Tone'] = ''
    new_escalated_df['Sentiment Analysis - Complaint 1 Case'] = ''
    new_escalated_df['Escalated Complaint (Tone Increased)'] = ''
    new_escalated_df['Tone Severity - Increased'] = ''
    new_escalated_df['Case Severity - Constant'] = ''
    new_escalated_df['Sentiment Analysis - Complaint 2 Tone'] = ''
    new_escalated_df['Sentiment Analysis - Complaint 2 Case'] = ''


    # Loop through the rows of subset
    for index, row in new_escalated_df.iterrows():
        if row['Tone Severity'] < 3 and row['Case Severity'] < 3:
            
            # Increase Case Severity
            task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity'] + 1} and a tone severity = {row['Tone Severity']}. "
            escalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
            new_escalated_df.at[index, 'Escalated Complaint (Case Increased)'] = escalated_complaint_case
            new_escalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
            new_escalated_df.at[index, 'Case Severity - Increased'] = row['Case Severity'] + 1


            # Increase Tone Severity
            task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] + 1}. "
            escalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
            new_escalated_df.at[index, 'Escalated Complaint (Tone Increased)'] = escalated_complaint_tone
            new_escalated_df.at[index, 'Tone Severity - Increased'] = row['Tone Severity'] + 1
            new_escalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']
        
        else:
            # Skip rows where either severity is already at maximum
            continue

    
    new_escalated_df.to_csv(f'{new_escalated_df}.csv', index=False)
    return new_escalated_df
    

# De-escalation code below: 

In [ ]:
def deescalate_complaint(dataframe):
     
     """Input: takes in a dataframe consisting of 3 columns: 
        - Original Complaint
        - Tone Severity
        - Case Severity
    
    Output: produces a csv file with synthetic complaints de-escalated by both case and tone severity
            returns the dataframe as well  
    """
     
    new_deescalated_df = dataframe.copy()
    # Initialize columns for de-escalated complaints
    new_deescalated_df['Sentiment Analysis - Original Tone'] = ''
    new_deescalated_df['Sentiment Analysis - Original Case'] = ''
    new_deescalated_df['De-escalated Complaint (Case Decreased)'] = ''
    new_deescalated_df['Tone Severity - Constant'] = ''
    new_deescalated_df['Case Severity - Decreased'] = ''
    new_deescalated_df['Sentiment Analysis - Complaint 1 Tone'] = ''
    new_deescalated_df['Sentiment Analysis - Complaint 1 Case'] = ''
    new_deescalated_df['De-escalated Complaint (Tone Decreased)'] = ''
    new_deescalated_df['Tone Severity - Decreased'] = ''
    new_deescalated_df['Case Severity - Constant'] = ''
    new_deescalated_df['Sentiment Analysis - Complaint 2 Tone'] = ''
    new_deescalated_df['Sentiment Analysis - Complaint 2 Case'] = ''

    # Loop through the rows of subset
    for index, row in new_deescalated_df.iterrows():
        if row['Tone Severity'] > 1 and row['Case Severity'] > 1:

            # Decrease Case Severity
            task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity'] - 1} and a tone severity = {row['Tone Severity']}. "
            deescalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
            new_deescalated_df.at[index, 'De-escalated Complaint (Case Decreased)'] = deescalated_complaint_case
            new_deescalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
            new_deescalated_df.at[index, 'Case Severity - Decreased'] = row['Case Severity'] - 1

            # Decrease Tone Severity
            task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] - 1}. "
            deescalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
            new_deescalated_df.at[index, 'De-escalated Complaint (Tone Decreased)'] = deescalated_complaint_tone
            new_deescalated_df.at[index, 'Tone Severity - Decreased'] = row['Tone Severity'] - 1
            new_deescalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']

        else:
        # Skip rows where de-escalation is not possible
            continue

    new_deescalated_df.to_csv(f'{new_deescalated_df}.csv', index=False)
    return new_deescalated_df

# Sentiment Analysis Section: 

YOU FIRST NEED TO CHANGE THE MODEL YOU ARE USING IN LM STUDIO

If you are using LlaMA for text gen, then perform sentiment analysis using Mistral and vice versa. 

In [27]:
def get_score(LLM_response: str, metric: str):
    """ 
    LLM_response: the sentiment analysis response of an LLM 
    and a metric as inputreturns the tone score as an integer 

    metric: takes in the strings: "tone" or "case"

    returns an integer score for desired metric
    """
    if metric == "tone":
        tone_match = re.search(r"Tone:\s*(\d+)", LLM_response)
        tone = int(tone_match.group(1)) if tone_match else None
        return tone

    elif metric == "case":
        case_match = re.search(r"Case:\s*(\d+)", LLM_response)
        case = int(case_match.group(1)) if case_match else None
        return case

    else: 
        error = 0
        return error


In [59]:
def analyze_sentiment(csv_name:str, transformation: str): 

    """
    Takes in a csv with generated complaints as input
    
    User inputs desired transformation: "Escalated", "De-escalated", "Duplicated"

    returns a new csv with the sentiment scores of those complaints
    """
    dataframe = pd.read_csv(csv_name) #load in csv as a pandas dataframe

    display(dataframe)

    transformations = {"Escalated" : "Increased", 
                       "De-escalated": "Decreased",
                        "Duplicated": "Constant"}
    
    complaint_columns = {'Original Complaint' : ['Sentiment Analysis - Original Tone', 'Sentiment Analysis - Original Case'] , 
                        
                        f"{transformation} Complaint (Case {transformations[transformation]})" : ['Sentiment Analysis - Complaint 1 Tone', 'Sentiment Analysis - Complaint 1 Case'],
                        f"{transformation} Complaint (Tone {transformations[transformation]})" : ['Sentiment Analysis - Complaint 2 Tone', 'Sentiment Analysis - Complaint 2 Case']
                        }

    columns = ['Original Complaint', 
               f"{transformation} Complaint (Case {transformations[transformation]})",  
               f"{transformation} Complaint (Tone {transformations[transformation]})"]

    for name in columns: 
        for index, row in dataframe.iterrows(): #would want to include dataframe here
                    
            
                role_sent_analysis = f"""You are an analyst at a commercial bank tasked with analyzing customer complaints. 
                The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
                case severity: severity of the their financial impact and tone severity: how harsh the complaint is. Your main priority is being efficient, concise, and objective. You ignore scores that
                may be present within the comment itself, and instead provide your own honest, objective score for both tone and case. You have been trained to follow this rubric: {rubric}. 
                Based on this criteria, provide a tone and case rating for the following complaint. {row[name]}

                Your response requires a simple format to follow or else you will agravvate your boss and risk losing your job. Your response should follow the following format: 

                "Tone: <insert score>
                Case: <insert score>

                <insert explanation>" """

                sentiment = chat_with_llm(role_sent_analysis)
                dataframe.at[index, complaint_columns[name][0]] = get_score(sentiment, "tone")
                # print(get_score(sentiment, "tone"))
                dataframe.at[index, complaint_columns[name][1]] = get_score(sentiment, "case")
                # print(get_score(sentiment, "case"))

    dataframe.to_csv(f'{csv_name}-Final.csv', index=False)
                
    return dataframe